In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score
wordnet_lemmatizer = WordNetLemmatizer()

In [ ]:
df = pd.read_csv('Tweets.csv')

In [ ]:
def normalizer(tweet):
      only_letters = re.sub("[^a-zA-Z]", " ", tweet)
      only_letters = only_letters.lower()
      only_letters = only_letters.split()
      filtered_result = [word for word in only_letters if word not in stopwords.words('english')]
      lemmas = [wordnet_lemmatizer.lemmatize(t) for t in filtered_result]
      lemmas = ' '.join(lemmas)
      return lemmas

In [ ]:
df = shuffle(df)
y = df['airline_sentiment']
x = df.text.apply(normalizer)

In [ ]:
vectorizer = CountVectorizer()
x_vectorized = vectorizer.fit_transform(x)

In [ ]:
train_x,val_x,train_y,val_y = train_test_split(x_vectorized,y)

In [ ]:
regressor = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model = regressor.fit(train_x, train_y)

In [ ]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
gs_clf = GridSearchCV(model, params, n_jobs=1, cv=5)
gs_clf = gs_clf.fit(train_x, train_y)
model = gs_clf.best_estimator_

In [ ]:
y_pred = model.predict(val_x)

_f1 = f1_score(val_y, y_pred, average='micro')
_confusion = confusion_matrix(val_y, y_pred)
__precision = precision_score(val_y, y_pred, average='micro')
_recall = recall_score(val_y, y_pred, average='micro')
_statistics = {'f1_score': _f1,
               'confusion_matrix': _confusion,
               'precision': __precision,
               'recall': _recall
              }

In [ ]:
print(_statistics)

{'f1_score': 0.7852459016393442, 'confusion_matrix': array([[2052,  203,   64],
       [ 260,  449,   71],
       [ 115,   73,  373]]), 'precision': 0.7852459016393443, 'recall': 0.7852459016393443}


In [ ]:
test_feature = vectorizer.transform(['Meat Week Day 3: Tummy hurts every night'])
model.predict(test_feature)

array(['negative'], dtype=object)

In [ ]:
test_feature = vectorizer.transform(['Movie is good'])
model.predict(test_feature)

array(['positive'], dtype=object)

In [ ]:
test_feature = vectorizer.transform(['I\'m okay'])
model.predict(test_feature)

array(['neutral'], dtype=object)

In [ ]:
import pickle

In [ ]:
pickl = {'vectorizer': vectorizer,
         'model': model
         }
pickle.dump(pickl, open('models'+".p", "wb"))